In [2]:
#Import Packages

import numpy as np
from numpy import random as rd
from pykrx import stock
import scipy as sp
import pandas as pd
from scipy import optimize

In [55]:
class Optima:
    def __init__(self):
        self.var_cov_mat = None
        self.rf = 0.015
        self.ret = None
        self.tickers = None
        self.data_start = None
        self.data_end = None
        self.N = None
        self.data = None
        
        self.tp = None # time Point
        self.windowLength = None
        
    def setTickers(self, tickers):
        self.tickers = tickers
        self.N = len(self.tickers)
    
    def setDataDate(self, srt, end):
        if type(srt) == str:
            self.data_start = srt
        else:
            self.data_start = str(srt)
        if type(end) == str:
            self.data_end = end
        else:
            self.data_end = str(end)
        
    def loadData(self):
        raw_data = []
        N = self.N
        for i in range(N):
            raw_data.append(stock.get_market_ohlcv_by_date(self.data_start, self.data_end, self.tickers[i])["종가"])
        self.data = pd.DataFrame(np.asarray(raw_data).T, columns = self.tickers, dtype=np.int32)
        self.ret = pd.DataFrame(np.nan, index=np.arange(self.data.shape[0]), columns=self.data.columns)
        for i in range(1, self.data.shape[0]):
            self.ret.iloc[i] = self.data.iloc[i]/self.data.iloc[i-1]
        self.var_cov_mat = np.identity(self.N)
    
    def gen_var_cov_mat(self):
        ret_tp = self.ret.iloc[self.tp - self.windowLength : self.tp]
        for i in range(0, self.N):
            for j in range(0, self.N):
                #print('-',end=' ')
                #print(np.cov(ret_tp.iloc[1:,i], ret_tp.iloc[1:,j])[0,1])
                self.var_cov_mat[i][j] = np.cov(ret_tp.iloc[1:,i], ret_tp.iloc[1:,j])[0,1]
    
    def setWindowLength(self, time):
        self.windowLength = time
    
    def setTimePoint(self, tp):
        self.tp = tp
        
    
    def eq_constraint(self, w):
        summation = 0
        for i in w:
            summation += i
            
        return summation - 1
    
    def flarray(self, w):
        ret_exd = np.cumprod(self.ret) - self.rf
        return -(np.dot(np.transpose(ret_exd.iloc[-1]), w) / (np.dot(np.dot(np.transpose(w), self.var_cov_mat), w))**(1/2))

    def Max_Sharp(self):
        return sp.optimize.fmin_slsqp(self.flarray, [1/self.N for i in range(self.N)], bounds=[(0,1) for i in range(self.N)], eqcons=[self.eq_constraint])
    
    
    def w(self):
        self.gen_var_cov_mat()
        return self.Max_Sharp()
    
    
    
    

In [57]:
optima = Optima()
tickers = ["002810", "003920", "004380", "011280"]#, "016580", "007340", "000370", "001510", "033270", "004710", "001500"]

optima.setTickers(tickers)
optima.setDataDate(20180101, 20191231)

optima.loadData()

optima.setWindowLength(30)
optima.setTimePoint(145)
optima.w()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -147.0135801155746
            Iterations: 8
            Function evaluations: 57
            Gradient evaluations: 8


array([0.39299318, 0.34043089, 0.07200914, 0.19456679])

In [59]:
optima.setWindowLength(29)
optima.w()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -145.4373672996802
            Iterations: 9
            Function evaluations: 61
            Gradient evaluations: 9


array([0.38168801, 0.35340861, 0.07573348, 0.1891699 ])

In [253]:
##############
for i in range((rebal.shape[0]//freq)):
    df = rebal.iloc[i*freq :(i+1)*freq]
    rebal['asdf'] = df
    
if rebal.shape[0]%freq != 0:
    print(rebal.iloc[-(rebal.shape[0]%freq):])


        0     1     2     3     4     5  asdf
110  1.06  0.91  1.01  0.92  1.05  0.93   NaN
111  0.99  0.94  0.97  0.90  0.92  0.91   NaN


In [221]:
rebal = pd.DataFrame(([1+np.random.randint(-10,12)*0.01 for i in range(6)] for i in range(112)))
rebal

,0,1,2,3,4,5
0,1.02,0.93,0.95,0.99,1.02,0.95
1,1.05,0.94,1.05,1.11,1.11,1.03
2,0.97,1.05,0.98,1.02,0.99,1.04
3,0.90,0.99,1.09,1.11,1.08,0.97
4,0.96,1.03,0.97,1.01,0.99,1.10
...,...,...,...,...,...,...
107,1.00,0.98,1.08,0.94,1.08,0.94
108,1.08,0.95,0.92,1.07,0.96,0.97
109,0.96,1.00,0.96,0.95,1.07,1.08
110,1.06,0.91,1.01,0.92,1.05,0.93


In [222]:
freq = 10
w = [1/5, 1/5, 1/5, 1/5, 1/5]

In [223]:
100-freq

90

In [234]:
133//10 * 10

130

In [233]:
133%10

3

In [239]:
rebal.shape[0]%freq

8

In [243]:
a = np.asarray([1,2,3])
b = np.asarray([1,2,3])
a*b

array([1, 4, 9])

In [242]:
for i in range((rebal.shape[0]//freq)):
    df = rebal.iloc[i*freq :(i+1)*freq]
    print(df)
    
if rebal.shape[0]%freq != 0:
    print(rebal.iloc[-(rebal.shape[0]%freq):])


      0     1     2     3     4     5
0  1.02  0.93  0.95  0.99  1.02  0.95
1  1.05  0.94  1.05  1.11  1.11  1.03
2  0.97  1.05  0.98  1.02  0.99  1.04
3  0.90  0.99  1.09  1.11  1.08  0.97
4  0.96  1.03  0.97  1.01  0.99  1.10
5  0.95  1.08  0.94  0.90  0.90  1.00
6  0.92  1.07  0.97  1.08  1.06  0.99
7  0.99  1.09  1.10  1.07  0.94  1.11
8  0.99  1.01  1.07  0.97  0.98  0.97
9  1.01  1.03  1.08  1.06  1.04  0.97
       0     1     2     3     4     5
10  1.09  0.93  1.03  1.07  1.01  0.94
11  0.97  1.04  1.02  0.91  0.91  1.04
12  0.93  1.09  0.93  0.91  0.97  1.00
13  1.04  0.97  1.00  1.06  1.03  1.05
14  1.01  0.90  0.98  1.01  1.02  0.91
15  0.99  0.90  1.05  1.08  1.01  1.08
16  1.04  0.91  1.04  0.98  0.96  0.93
17  1.00  1.07  1.10  0.93  0.96  0.94
18  1.11  1.10  0.97  0.98  0.92  0.91
19  1.06  1.02  0.92  1.03  1.07  1.09
       0     1     2     3     4     5
20  0.93  0.98  1.00  1.01  1.07  0.90
21  1.11  0.95  1.08  0.95  1.11  0.94
22  0.93  0.95  0.96  0.98  1.00  1.

In [119]:
rebal.iloc[1:5].cumprod()

,0,1,2,3,4,5
1,1.070000,0.900000,0.900000,1.020000,1.030000,0.900000
2,0.984400,0.882000,0.810000,0.948600,0.999100,0.819000
3,1.092684,0.943740,0.801900,1.033974,1.059046,0.761670
4,1.212879,0.972052,0.745767,0.982275,1.016684,0.845454


In [120]:
rebal.iloc[5:10].cumprod()

,0,1,2,3,4,5
5,1.060000,1.100000,0.910000,1.04000,0.960000,1.000000
6,1.134200,1.133000,0.819000,0.98800,0.892800,0.940000
7,1.100174,1.178320,0.794430,1.07692,0.964224,0.921200
8,1.045165,1.225453,0.714987,1.07692,0.896728,0.985684
9,1.076520,1.127417,0.779336,1.07692,0.941565,0.965970


In [121]:
rebal.head(11)

,0,1,2,3,4,5
0,1.10,1.07,0.93,0.98,0.97,1.07
1,1.07,0.90,0.90,1.02,1.03,0.90
2,0.92,0.98,0.90,0.93,0.97,0.91
3,1.11,1.07,0.99,1.09,1.06,0.93
4,1.11,1.03,0.93,0.95,0.96,1.11
5,1.06,1.10,0.91,1.04,0.96,1.00
6,1.07,1.03,0.90,0.95,0.93,0.94
7,0.97,1.04,0.97,1.09,1.08,0.98
8,0.95,1.04,0.90,1.00,0.93,1.07
9,1.03,0.92,1.09,1.00,1.05,0.98


In [156]:
pd.DataFrame.cumsum(rebal.cumprod().iloc[-1])[-1:]/5

5    2.372722
Name: 99, dtype: float64

In [134]:
rebal.cumprod().iloc[-1]

0    3.541576
1    0.591426
2    0.583374
3    1.051113
4    4.484368
5    1.611752
Name: 99, dtype: float64